<a href="https://colab.research.google.com/github/Taro489/Tugas_IndividuBigData/blob/master/DATAPROC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA PROCESSING

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark_dist_explore

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [3]:
import findspark
findspark.init("spark-2.4.6-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark_dist_explore import Histogram, hist

In [5]:
from collections import Counter
import pandas as pd
import seaborn as sn
import numpy as np
import csv

In [7]:
sc = spark.sparkContext
spark = SparkSession(sc)


spotify_df = spark.read.csv("datasets_456949_861300_top10s.csv", header=True, inferSchema=True)
spotify_df.printSchema() 

root
 |-- _c0: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- top genre: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- bpm: integer (nullable = true)
 |-- nrgy: integer (nullable = true)
 |-- dnce: integer (nullable = true)
 |-- dB: integer (nullable = true)
 |-- live: integer (nullable = true)
 |-- val: integer (nullable = true)
 |-- dur: integer (nullable = true)
 |-- acous: integer (nullable = true)
 |-- spch: integer (nullable = true)
 |-- pop: integer (nullable = true)



In [8]:
spotify_df.show(3)

+---+--------------------+------+---------------+----+---+----+----+---+----+---+---+-----+----+---+
|_c0|               title|artist|      top genre|year|bpm|nrgy|dnce| dB|live|val|dur|acous|spch|pop|
+---+--------------------+------+---------------+----+---+----+----+---+----+---+---+-----+----+---+
|  1|    Hey, Soul Sister| Train|     neo mellow|2010| 97|  89|  67| -4|   8| 80|217|   19|   4| 83|
|  2|Love The Way You Lie|Eminem|detroit hip hop|2010| 87|  93|  75| -5|  52| 64|263|   24|  23| 82|
|  3|             TiK ToK| Kesha|      dance pop|2010|120|  84|  76| -3|  29| 71|200|   10|  14| 80|
+---+--------------------+------+---------------+----+---+----+----+---+----+---+---+-----+----+---+
only showing top 3 rows



In [ ]:
spotify_df.groupby("top genre").count().sort(col("count").desc()).show(100, False)

+-------------------------+-----+
|top genre                |count|
+-------------------------+-----+
|dance pop                |327  |
|pop                      |60   |
|canadian pop             |34   |
|barbadian pop            |15   |
|boy band                 |15   |
|electropop               |13   |
|british soul             |11   |
|big room                 |10   |
|canadian contemporary r&b|9    |
|neo mellow               |9    |
|art pop                  |8    |
|complextro               |6    |
|hip pop                  |6    |
|australian dance         |6    |
|atl hip hop              |5    |
|australian pop           |5    |
|edm                      |5    |
|latin                    |4    |
|hip hop                  |4    |
|permanent wave           |4    |
|colombian pop            |3    |
|tropical house           |3    |
|folk-pop                 |2    |
|belgian edm              |2    |
|escape room              |2    |
|acoustic pop             |2    |
|canadian hip 

Seleksi data yang digunakan dan ditulis lagi kedalam csv yang baru

In [10]:
spotify_df1 = spotify_df.withColumnRenamed("top genre", "genre")
spotify_df1.createOrReplaceTempView("songs")
df1 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2010 group by genre''')
df1.coalesce(1).write.csv("/content/df1.csv")

In [25]:
spotify_genre = spotify_df.groupby("top genre","year")
spotify_genre.count().filter("year = 2010").show()

+---------------+----+-----+
|      top genre|year|count|
+---------------+----+-----+
|        hip pop|2010|    3|
|     neo mellow|2010|    1|
|      indie pop|2010|    1|
|    atl hip hop|2010|    3|
|  barbadian pop|2010|    2|
|      dance pop|2010|   31|
|   canadian pop|2010|    1|
|detroit hip hop|2010|    1|
|       big room|2010|    1|
|  colombian pop|2010|    1|
|        art pop|2010|    1|
| australian pop|2010|    2|
|            pop|2010|    3|
+---------------+----+-----+



In [15]:
df2 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2011 group by genre''')
df2.coalesce(1).write.csv("/content/df2")

In [27]:
spotify_genre.count().filter("year = 2011").show()

+--------------+----+-----+
|     top genre|year|count|
+--------------+----+-----+
|     dance pop|2011|   38|
|  british soul|2011|    2|
|           pop|2011|    4|
|   atl hip hop|2011|    1|
|  canadian pop|2011|    1|
|  acoustic pop|2011|    1|
|australian pop|2011|    1|
|   chicago rap|2011|    1|
| barbadian pop|2011|    4|
+--------------+----+-----+



In [16]:
df3 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2012 group by genre''')
df3.coalesce(1).write.csv("/content/df3")

In [28]:
spotify_genre.count().filter("year = 2012").show()

+--------------+----+-----+
|     top genre|year|count|
+--------------+----+-----+
|  british soul|2012|    1|
|           pop|2012|    7|
|   baroque pop|2012|    2|
|     dance pop|2012|   15|
| barbadian pop|2012|    3|
|  canadian pop|2012|    3|
|permanent wave|2012|    1|
|      boy band|2012|    2|
|australian pop|2012|    1|
+--------------+----+-----+



In [17]:
df4 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2013 group by genre''')
df4.coalesce(1).write.csv("/content/df4")

In [29]:
spotify_genre.count().filter("year = 2013").show()

+----------------+----+-----+
|       top genre|year|count|
+----------------+----+-----+
|     celtic rock|2013|    1|
|             pop|2013|    5|
|        big room|2013|    2|
|         electro|2013|    2|
|   barbadian pop|2013|    1|
|    acoustic pop|2013|    1|
|  permanent wave|2013|    1|
|     atl hip hop|2013|    1|
|         hip pop|2013|    1|
|       indie pop|2013|    1|
|      complextro|2013|    1|
|       dance pop|2013|   42|
|         art pop|2013|    1|
|canadian hip hop|2013|    1|
|        boy band|2013|    6|
|    alaska indie|2013|    1|
|       candy pop|2013|    1|
|    canadian pop|2013|    2|
+----------------+----+-----+



In [18]:
df5 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2014 group by genre''')
df5.coalesce(1).write.csv("/content/df5")

In [30]:
spotify_genre.count().filter("year = 2014").show()

+------------------+----+-----+
|         top genre|year|count|
+------------------+----+-----+
|     colombian pop|2014|    2|
|               pop|2014|    9|
|          folk-pop|2014|    1|
|        neo mellow|2014|    6|
|    permanent wave|2014|    1|
|          boy band|2014|    1|
|        electropop|2014|    1|
|  australian dance|2014|    1|
|      metropopolis|2014|    1|
|           hip pop|2014|    1|
|             house|2014|    1|
|          big room|2014|    1|
|australian hip hop|2014|    1|
|        complextro|2014|    1|
|         hollywood|2014|    1|
|         candy pop|2014|    1|
|         dance pop|2014|   27|
|           art pop|2014|    1|
+------------------+----+-----+



In [19]:
df6 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2015 group by genre''')
df6.coalesce(1).write.csv("/content/df6")

In [31]:
spotify_genre.count().filter("year = 2015").show()

+--------------------+----+-----+
|           top genre|year|count|
+--------------------+----+-----+
|        british soul|2015|    2|
|          complextro|2015|    1|
|      australian pop|2015|    1|
|        canadian pop|2015|   11|
|       barbadian pop|2015|    2|
|            boy band|2015|    3|
|           dance pop|2015|   52|
|            big room|2015|    2|
|                 pop|2015|    8|
|    australian dance|2015|    2|
|irish singer-song...|2015|    1|
|    french indie pop|2015|    1|
|             hip hop|2015|    1|
|canadian contempo...|2015|    3|
|      tropical house|2015|    2|
|             hip pop|2015|    1|
|         belgian edm|2015|    1|
|             art pop|2015|    1|
+--------------------+----+-----+



In [20]:
df7 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2016 group by genre''')
df7.coalesce(1).write.csv("/content/df7")

In [32]:
spotify_genre.count().filter("year = 2016").show()

+--------------------+----+-----+
|           top genre|year|count|
+--------------------+----+-----+
|             hip hop|2016|    1|
|       barbadian pop|2016|    2|
|                 pop|2016|    1|
|          neo mellow|2016|    2|
|    canadian hip hop|2016|    1|
|        canadian pop|2016|    7|
|canadian contempo...|2016|    3|
|           dance pop|2016|   46|
|          electropop|2016|    4|
|          complextro|2016|    1|
|            big room|2016|    1|
|      canadian latin|2016|    1|
|            folk-pop|2016|    1|
|      tropical house|2016|    1|
|    australian dance|2016|    1|
|        british soul|2016|    4|
|             art pop|2016|    1|
|          danish pop|2016|    1|
|               latin|2016|    1|
+--------------------+----+-----+



In [21]:
df8 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2017 group by genre''')
df8.coalesce(1).write.csv("/content/df8")

In [33]:
spotify_genre.count().filter("year = 2017").show()

+--------------------+----+-----+
|           top genre|year|count|
+--------------------+----+-----+
|canadian contempo...|2017|    3|
|          complextro|2017|    2|
|       electro house|2017|    1|
|      permanent wave|2017|    1|
|           dance pop|2017|   31|
|                 pop|2017|    5|
|               latin|2017|    1|
|        british soul|2017|    2|
|            big room|2017|    2|
|        canadian pop|2017|    3|
|             art pop|2017|    3|
|           downtempo|2017|    1|
|          electropop|2017|    3|
|         belgian edm|2017|    1|
|            boy band|2017|    1|
|    australian dance|2017|    2|
|     electronic trap|2017|    1|
|                 edm|2017|    2|
+--------------------+----+-----+



In [22]:
df9 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2018 group by genre''')
df9.coalesce(1).write.csv("/content/df9")

In [34]:
spotify_genre.count().filter("year = 2018").show()

+--------------------+----+-----+
|           top genre|year|count|
+--------------------+----+-----+
|contemporary country|2018|    1|
|                 edm|2018|    2|
|          electropop|2018|    1|
|               latin|2018|    1|
|                 pop|2018|    9|
|     detroit hip hop|2018|    1|
|             hip hop|2018|    2|
|           dance pop|2018|   38|
|        canadian pop|2018|    5|
|           downtempo|2018|    1|
|             brostep|2018|    1|
|       barbadian pop|2018|    1|
|        moroccan pop|2018|    1|
+--------------------+----+-----+



In [23]:
df10 = spark.sql('''select genre, COUNT(genre) category_count from songs WHERE year = 2019 group by genre''')
df10.coalesce(1).write.csv("/content/df10")

In [35]:
spotify_genre.count().filter("year = 2019").show()

+---------------+----+-----+
|      top genre|year|count|
+---------------+----+-----+
|electronic trap|2019|    1|
|     electropop|2019|    4|
|       boy band|2019|    2|
|   canadian pop|2019|    1|
|        brostep|2019|    1|
|            edm|2019|    1|
|alternative r&b|2019|    1|
|          latin|2019|    1|
|    escape room|2019|    2|
|      dance pop|2019|    7|
|            pop|2019|    9|
|       big room|2019|    1|
+---------------+----+-----+

